In [ ]:
# INSTALL REQUIRED PACKAGES AND MAKE IMPORTS

#!pip install googlemaps  # IF NOT ALREADY INSTALLED

import googlemaps
import pandas

In [ ]:
# INITIALIZE GOOGLE CLOUD SERVICE

GOOGLE_MAPS_API_KEY = " <INSERT YOUR API KEY HERE> "

# Initiate Google Maps Platform API
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)


In [ ]:
# DECLARE STARTPOINT(s) AND DESTINATION(s)

#        ---STARTPOINTS---
# Startpoints
startpoint1 = (-1.1374252134825595,36.91097136121056)     # SP1
startpoint2 = (-1.1510145675699641,36.642598387166494)    # SP2
startpoint3 = (-1.1201086824168112,36.914511961509476)    # SP3

#         ---DESTINATIONS---
# Declare destinations
endpoint = (-1.215808447681381,36.99104601796648)         # EP
endpoint2 = (-1.2434231212273734,36.87341187999162)       # EP2

#         ---MULTIPLE DESTINATIONS---
# Import csv file with a list of (lat, long) coordinates

# 1) ---jupyternotebook---
coordinate_data = pandas.read_csv(" <Filename> ")

# 2) ---Google Colab---
from google.colab import files
uploaded = files.upload() # This will prompt a file upload from your drive

#  Create dataframe from the imported CSV files

# 1) ---jupyternotebook---
df = coordinate_data

# 2) ---Google Colab---
df = pandas.read_csv(io.BytesIO(uploaded[' <Filename of uploaded file> ']))

df = df.dropna() # Drop empty rows
df.head(5)

In [ ]:
## USING GOOGLEMAPS DISTANCE MATRIX API TO ESTIMATE TIME AND DISTANCE FROM A POINT TO A POINT

result = gmaps.distance_matrix(origins = (startpoint1), destinations = (endpoint), mode='driving')

distresult = result["rows"][0]["elements"][0]["distance"]["value"]/1000   # Convert to KM by /1000
timeresult = result["rows"][0]["elements"][0]["duration"]["value"]/60     # Convert to Minutes by /60

print("Distance (KM) SP1 to EP: " , round(distresult,1), '\n', "Time (MINS) SP1 to EP: ", round(timeresult,0))



In [ ]:
# USING GOOGLEMAPS DISTANCE MATRIX API TO ESTIMATE TIME AND DISTANCE FROM MULTIPLE POINTS TO A POINT.

# List the startpoints in the origins variable - (lat,long),(lat,long),(lat,long)...
result = gmaps.distance_matrix(origins = [(startpoint1),(startpoint2),(startpoint3)], destinations=(endpoint2), mode='driving')

dist_SP1_to_EP2 = round(result["rows"][0]["elements"][0]["distance"]["value"]/1000,1)
dist_SP2_to_EP2 = round(result["rows"][1]["elements"][0]["distance"]["value"]/1000,1)
dist_SP3_to_EP2 = round(result["rows"][2]["elements"][0]["distance"]["value"]/1000,1)

time_SP1_to_EP2 = round(result["rows"][0]["elements"][0]["duration"]["value"]/60,0)
time_SP2_to_EP2 = round(result["rows"][1]["elements"][0]["duration"]["value"]/60,0)
time_SP3_to_EP2 = round(result["rows"][2]["elements"][0]["duration"]["value"]/60,0)

print('\n', "Startpoint1 to endpoint2:", '\n', "Distance (KM): " , dist_SP1_to_EP2, '   ', "Time (MINS): ", time_SP1_to_EP2, '\n',
     '\n', "Startpoint2 to endpoint2:", '\n', "Distance (KM): " , dist_SP2_to_EP2, '   ', "Time (MINS): ", time_SP2_to_EP2, '\n',
     '\n', "Startpoint3 to endpoint2:", '\n', "Distance (KM): " , dist_SP3_to_EP2, '   ', "Time (MINS): ", time_SP3_to_EP2, '\n')

In [ ]:
# USING GOOGLEMAPS DISTANCE MATRIX API TO ESTIMATE TIME AND DISTANCE FROM MULTIPLE POINTS TO MULTIPLE POINTS.

# List the startpoints and destinations in the origins variable - (lat,long),(lat,long),(lat,long)...
result = gmaps.distance_matrix(origins = [(startpoint1),(startpoint2)], destinations=[(endpoint),(endpoint2)], mode='driving')

dist_SP1_to_EP = round(result["rows"][0]["elements"][0]["distance"]["value"]/1000,1)
dist_SP1_to_EP2 = round(result["rows"][0]["elements"][1]["distance"]["value"]/1000,1)
dist_SP2_to_EP = round(result["rows"][1]["elements"][0]["distance"]["value"]/1000,1)
dist_SP2_to_EP2 = round(result["rows"][1]["elements"][1]["distance"]["value"]/1000,1)

time_SP1_to_EP = round(result["rows"][0]["elements"][0]["duration"]["value"]/60,0)
time_SP1_to_EP2 = round(result["rows"][0]["elements"][1]["duration"]["value"]/60,0)
time_SP2_to_EP = round(result["rows"][1]["elements"][0]["duration"]["value"]/60,0)
time_SP2_to_EP2 = round(result["rows"][1]["elements"][1]["duration"]["value"]/60,0)

print('\n', "Startpoint1 to endpoint:", '\n', "Distance (KM): " , dist_SP1_to_EP, '   ', "Time (MINS): ", time_SP1_to_EP, '\n',
     '\n', "Startpoint1 to endpoint2:", '\n', "Distance (KM): " , dist_SP1_to_EP2, '   ', "Time (MINS): ", time_SP1_to_EP2, '\n',
     '\n', "Startpoint2 to endpoint:", '\n', "Distance (KM): " , dist_SP2_to_EP, '   ', "Time (MINS): ", time_SP2_to_EP, '\n',
     '\n', "Startpoint2 to endpoint2:", '\n', "Distance (KM): " , dist_SP2_to_EP2, '   ', "Time (MINS): ", time_SP2_to_EP2, '\n')

In [ ]:
# USING GOOGLEMAPS DISTANCE MATRIX API TO ESTIMATE TIME AND DISTANCE FROM A POINT TO A LIST OF MANY POINTS.

# Declare the column holding the coordinates (lat,long)
destinations= df.COORDS # 'COORDS' is the column name containing (lat,long) values

# Declare empty arrays for the distance and time values
DISTANCES = []
TIMES = []

# Iterate through the rows of the dataframe saving the distance and time estimates for each to the coordinate points

for destination in destinations:
    result = gmaps.distance_matrix(origins = (startpoint2), destinations = (destination), mode='driving')
    
# Check the status of the result to prevent zero_result from raising key errors
    status = result["rows"][0]["elements"][0]["status"]
    
# Save the value if status is okay
    if status == "OK":
        distresult = result["rows"][0]["elements"][0]["distance"]["value"]/1000
        timeresult = result["rows"][0]["elements"][0]["duration"]["value"]/60
        
        DISTANCES.append(round(distresult,1))
        TIMES.append(round(timeresult,0))
 # if not, set the value to -404 to keep type as int and distinguish from same location which has 0 distance and time
    else:
        dist = -404
        time = -404
        
        DISTANCES.append(dist)
        TIMES.append(time)

# Create new columns in the dataframe to store the distance and time estimate values

df["time_SP2(MINS)"] = TIMES
df["distance_SP2(KM)"] = DISTANCES

# Check first 5 rows
df.head(5)

# EXPORT DATAFRAME AS CSV

df.to_csv('df.csv')       # Saves to current working directory for jupyter -- alternatively, add path to desired directory
files.download('df.csv')  # ---Google Colab--- to downoad the file to your local machine

In [ ]:
# USING GOOGLEMAPS DISTANCE MATRIX API TO ESTIMATE TIME AND DISTANCE FROM MULTIPLE POINTS TO A LIST OF MANY POINTS.

# Declare the column holding the coordinates (lat,long)
destinations= df.COORDS # 'COORDS' is the column name containing (lat,long) values

# Declare empty arrays for the distance and time values with regard to each starting point
DISTANCES_SP1 = []
TIMES_SP1 = []
DISTANCES_SP2 = []
TIMES_SP2 = []
DISTANCES_SP3 = []
TIMES_SP3 = []

# function to append data to empty arrays based on API result

def append_data(result, status, i, DISTANCES, TIMES):
    # Save the value if status is okay
    if status == "OK":
        distresult = result["rows"][i]["elements"][0]["distance"]["value"]/1000
        DISTANCES.append(round(distresult,1))
        
        timeresult = result["rows"][i]["elements"][0]["duration"]["value"]/60
        TIMES.append(round(timeresult,0))
        
     # if not, set the value to -404 to keep type as int and distinguish from same location which has 0 distance and time
    else:
        dist = -404
        time = -404
        DISTANCES.append(dist)
        TIMES.append(time)
    
# Iterate through the rows of the dataframe saving the distance and time estimates for each to the coordinate points

for destination in destinations:
    result = gmaps.distance_matrix(origins = [(startpoint1),(startpoint2),(startpoint3)], destinations=(destination), mode='driving')

    # Check the status of the result to prevent zero_result from raising key errors
    status1 = result["rows"][0]["elements"][0]["status"]
    status2 = result["rows"][1]["elements"][0]["status"]
    status3 = result["rows"][2]["elements"][0]["status"]
    
    # Append distance and time data to empty arrays for each startpoint
    append_data(result, status1, 0, DISTANCES_SP1, TIMES_SP1)
    append_data(result, status2, 1, DISTANCES_SP2, TIMES_SP2)
    append_data(result, status3, 2, DISTANCES_SP3, TIMES_SP3)
    

# Create new columns in the dataframe to store the distance and time estimate values

df["time_SP1(MINS)"] = TIMES_SP1
df["distance_SP1(KM)"] = DISTANCES_SP1
df["time_SP2(MINS)"] = TIMES_SP2
df["distance_SP2(KM)"] = DISTANCES_SP2
df["time_SP3(MINS)"] = TIMES_SP3
df["distance_SP3(KM)"] = DISTANCES_SP3

# Check first 5 rows
df.head(5)

# EXPORT DATAFRAME AS CSV

df.to_csv('df.csv')       # Saves to current working directory for jupyter -- alternatively, add path to desired directory
files.download('df.csv')  # ---Google Colab--- to downoad the file to your local machine